# Проект по анализу базы данных сервиса для чтения книг по подписке (SQL)

Цель — проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'password', # пароль
'host': 'host',
'port': port, # порт подключения
'db': 'db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Изучение общей информации 

Изучу данные в таблицах

In [2]:
query = (
'''
    SELECT *
    FROM books

''')

pd.io.sql.read_sql(query, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = (
'''
    SELECT *
    FROM authors

''')

pd.io.sql.read_sql(query, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = (
'''
    SELECT *
    FROM publishers

''')

pd.io.sql.read_sql(query, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = (
'''
    SELECT *
    FROM ratings

''')

pd.io.sql.read_sql(query, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = (
'''
    SELECT *
    FROM reviews

''')

pd.io.sql.read_sql(query, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## SQL запросы и выводы

Посчитаю, сколько книг вышло после 1 января 2000 года.

In [7]:
query = (
'''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01'

''')

pd.io.sql.read_sql(query, con = engine).head()

,count
0,819


Вывод: После 1 января 2000 года вышло 819 книг.

Для каждой книги посчитаю количество обзоров и среднюю оценку.

In [8]:
query = (
'''
    SELECT b.book_id, b.title, c.review_quantity, d.average_rating
    FROM books AS b
    LEFT OUTER JOIN
        (SELECT book_id, COUNT(review_id) AS review_quantity
        FROM reviews
        GROUP BY book_id
        ORDER BY book_id ASC) AS c ON b.book_id = c.book_id
    LEFT OUTER JOIN
        (SELECT book_id, AVG(rating) AS average_rating
        FROM ratings
        GROUP BY book_id
        ORDER BY book_id ASC) AS d ON b.book_id = d.book_id
''')
pd.io.sql.read_sql(query, con = engine).head()

,book_id,title,review_quantity,average_rating
0,1,'Salem's Lot,2.0,3.666667
1,2,1 000 Places to See Before You Die,1.0,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
3,4,1491: New Revelations of the Americas Before C...,2.0,4.500000
4,5,1776,4.0,4.000000


Вывод: Получила таблицу с количеством обзоров и средней оценкой по каждой книге.

Определю издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключу из анализа брошюры.

In [9]:
query = (
'''
    SELECT p.publisher, COUNT(book_id) AS number_of_books
    FROM books AS b
    INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY number_of_books DESC
    LIMIT 1
''')
pd.io.sql.read_sql(query, con = engine)

,publisher,number_of_books
0,Penguin Books,42


Вывод: Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books.

Определю автора с самой высокой средней оценкой книг — буду учитывать только книги с 50 и более оценками.

In [10]:
query = (
'''
    SELECT author, AVG(rating) AS average_rating
    FROM
        (SELECT book_id, COUNT(rating_id) AS number_of_rates
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) > 50) AS c
    LEFT OUTER JOIN 
        (SELECT book_id, author, b.author_id
        FROM books AS b
        LEFT OUTER JOIN authors AS a ON b.author_id = a.author_id) AS d ON c.book_id = d.book_id
    LEFT OUTER JOIN ratings AS r ON r.book_id = c.book_id
    GROUP BY author
    ORDER BY average_rating DESC
    LIMIT 1
    
''')
pd.io.sql.read_sql(query, con = engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


 Вывод: Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré.

Посчитаю среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
query = (
'''
    SELECT AVG(number_of_reviews)
    FROM
        (SELECT a.username, COUNT(review_id) AS number_of_reviews
        FROM
            (SELECT username, COUNT(rating) AS number_of_rates
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50) AS a
        LEFT OUTER JOIN reviews AS b ON a.username = b.username
        GROUP BY a.username) AS d
''')
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Вывод: Среднее количество обзоров от пользователей, которые поставили больше 50 оценок ~ 24,3.